#



In [21]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import pandas_datareader as web
import datetime
import yfinance as yf

In [23]:
# set up which stock we want to analysis
alstom = "ALO.PA"
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now().date()

## Step1. Get the stock historical data

There are many ways to get the stock data:
- yfinance: test success
- tushare: test failed
- pandas_datareader [pandas-datareader](https://pydata.github.io/pandas-datareader/): test failed


In [19]:
# test datareader failed
df = web.DataReader("TSLA", 'yahoo', start, end)

TypeError: string indices must be integers

In [24]:
alstom_df=yf.download(alstom,start,end)

[*********************100%***********************]  1 of 1 completed


### 1.1 Explore the data

In [25]:
# show the first 5 lines
alstom_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,26.900000,27.100000,26.650000,26.875000,22.656872,676180
2015-01-05,26.705000,27.020000,26.129999,26.200001,22.087814,1092679
2015-01-06,26.145000,26.655001,25.955000,26.215000,22.100460,1281398
2015-01-07,26.250000,26.775000,25.955000,26.525000,22.361805,997854
2015-01-08,26.809999,27.215000,26.730000,27.004999,22.766466,1256792


In [26]:
# show the last 5 lines
alstom_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-08-02,26.879999,27.100000,26.549999,26.910000,26.910000,1074841
2023-08-03,26.600000,26.780001,26.379999,26.559999,26.559999,692961
2023-08-04,26.629999,26.879999,26.330000,26.719999,26.719999,633930
2023-08-08,26.200001,26.350000,25.740000,26.000000,26.000000,754418
2023-08-09,26.370001,26.580000,26.139999,26.139999,26.139999,676471


In [27]:
alstom_df.shape

(2203, 6)

In [28]:
# get the column names
print(alstom_df.columns)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


In [29]:
# this shows the schema of the dataframe
alstom_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2203 entries, 2015-01-02 to 2023-08-09
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2203 non-null   float64
 1   High       2203 non-null   float64
 2   Low        2203 non-null   float64
 3   Close      2203 non-null   float64
 4   Adj Close  2203 non-null   float64
 5   Volume     2203 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 120.5 KB


> Unlike spark df, the pandas df has the notion of index. In the above example, the index is date column which has type Datetime

## Step 2. Basic pandas time series functions

In section 3 EDA, we will use some pandas functions which you may not familiar, here we examine them first.
The pandas dataframe provide a list of useful functions which can help us to explore the data
- rolling
- ewm (Exponentially Weighted Moving)


### 2.1 Rolling function

The rolling function is used for calculating `rolling statistics` over a specified window of data.

Below query will calculate the 7-day rolling average of the closing price

In [30]:
rolling_avg = alstom_df["Close"].rolling(window=7).mean()

In [31]:
rolling_avg.head(10)

Date
2015-01-02          NaN
2015-01-05          NaN
2015-01-06          NaN
2015-01-07          NaN
2015-01-08          NaN
2015-01-09          NaN
2015-01-12    26.660000
2015-01-13    26.726429
2015-01-14    26.813571
2015-01-15    26.910714
Name: Close, dtype: float64

There are two things that worth to mention:
1. The first 6 days have `NaN values` since there is not enough data to calculate a 7-day rolling average. Starting from the 7th day, it will show the rolling average for each subsequent day.
2. The rolling is based on the index, as we use date as index, so window = 7 means 7 day. If the index is year such as 2010, 2011, it will be interpreted as 7 years

## 2.2 Exponentially Weighted Moving

This function is often used to calculate exponentially weighted statistics for a given data series. It is commonly used for smoothing and analyzing `time series data`.

**Exponentially weighted moving averages give more weight to recent observations while gradually decreasing the weight of older observations. This can be useful for capturing trends and patterns in data while reducing the impact of random fluctuations or noise.**

In [32]:
alstom_df['ewm'] = alstom_df['Adj Close'].ewm(span=20,
                                              min_periods=0,
                                              adjust=False,
                                              ignore_na=False).mean()

In [33]:
alstom_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,ewm
Date,,,,,,,
2015-01-02,26.900000,27.100000,26.650000,26.875000,22.656872,676180,22.656872
2015-01-05,26.705000,27.020000,26.129999,26.200001,22.087814,1092679,22.602676
2015-01-06,26.145000,26.655001,25.955000,26.215000,22.100460,1281398,22.554846
2015-01-07,26.250000,26.775000,25.955000,26.525000,22.361805,997854,22.536461
2015-01-08,26.809999,27.215000,26.730000,27.004999,22.766466,1256792,22.558366
2015-01-09,27.000000,27.174999,26.705000,26.825001,22.614721,667945,22.563733
2015-01-12,26.879999,27.260000,26.785000,26.975000,22.741175,632136,22.580633
2015-01-13,26.930000,27.645000,26.840000,27.340000,23.048887,975225,22.625228
2015-01-14,27.010000,27.225000,26.610001,26.809999,22.602074,993274,22.623023


The **span** parameter is called the `smoothing parameter`. It defines the weight difference of more recent and less recent observations. The formula of the weight calculation is : **Weight = 2 / (span + 1)**

For example, if we specify `span=3`, it means that the most recent observation is given the highest weight (e.g., approximately 0.5), the observation before that is given a lower weight (e.g., approximately 0.25), and the observation before that is given an even lower weight (e.g., approximately 0.125).

The **number of observations included in the calculation** of the EWMA is calculated with below formula:
**Number of Observations = 2 * span - 1**

In our example, we take span =20, so we have `Number of Observations = 2 * 20 - 1 = 39`